In [1]:
# Sample dataset with facts about Berlin
documents = [
    "Berlin is the capital and largest city of Germany by both area and population.",
    "Berlin is known for its art scene and modern landmarks like the Berliner Philharmonie.",
    "The Berlin Wall, which divided the city from 1961 to 1989, was a significant Cold War symbol.",
    "Berlin has more bridges than Venice, with around 1,700 bridges.",
    "The city's Zoological Garden is the most visited zoo in Europe and one of the most popular worldwide.",
    "Berlin's Museum Island is a UNESCO World Heritage site with five world-renowned museums.",
    "The Reichstag building houses the German Bundestag (Federal Parliament).",
    "Berlin is famous for its diverse architecture, ranging from historic buildings to modern structures.",
    "The Berlin Marathon is one of the world's largest and most popular marathons.",
    "Berlin's public transportation system includes buses, trams, U-Bahn (subway), and S-Bahn (commuter train).",
    "The Brandenburg Gate is an iconic neoclassical monument in Berlin.",
    "Berlin has a thriving startup ecosystem and is considered a major tech hub in Europe.",
    "The city hosts the Berlinale, one of the most prestigious international film festivals.",
    "Berlin has more than 180 kilometers of navigable waterways.",
    "The East Side Gallery is an open-air gallery on a remaining section of the Berlin Wall.",
    "Berlin's Tempelhofer Feld, a former airport, is now a public park and recreational area.",
    "The TV Tower at Alexanderplatz offers panoramic views of the city.",
    "Berlin's Tiergarten is one of the largest urban parks in Germany.",
    "Checkpoint Charlie was a famous crossing point between East and West Berlin during the Cold War.",
    "Berlin is home to numerous theaters, including the Berliner Ensemble and the Volksbühne.",
    "The Berlin Philharmonic Orchestra is one of the most famous orchestras in the world.",
    "Berlin has a vibrant nightlife scene, with countless bars, clubs, and music venues.",
    "The Berlin Cathedral is a major Protestant church and a landmark of the city.",
    "Charlottenburg Palace is the largest palace in Berlin and a major tourist attraction.",
    "Berlin's Alexanderplatz is a large public square and transport hub in central Berlin.",
    "Berlin is known for its street art, with many murals and graffiti artworks around the city.",
    "The Gendarmenmarkt is a historic square in Berlin featuring the Konzerthaus, French Cathedral, and German Cathedral.",
    "Berlin has a strong coffee culture, with numerous cafés throughout the city.",
    "The Berlin TV Tower is the tallest structure in Germany, standing at 368 meters.",
    "Berlin's KaDeWe is one of the largest and most famous department stores in Europe.",
    "The Berlin U-Bahn network has 10 lines and serves 173 stations.",
    "Berlin has a population of over 3.6 million people.",
    "The city of Berlin covers an area of 891.8 square kilometers.",
    "Berlin has a temperate seasonal climate.",
    "The Berlin International Film Festival, also known as the Berlinale, is one of the world's leading film festivals.",
    "Berlin is home to the Humboldt University, founded in 1810.",
    "The Berlin Hauptbahnhof is the largest train station in Europe.",
    "Berlin's Tegel Airport closed in 2020, and operations moved to Berlin Brandenburg Airport.",
    "The Spree River runs through the center of Berlin.",
    "Berlin is twinned with Los Angeles, California, USA.",
    "The Berlin Botanical Garden is one of the largest and most important botanical gardens in the world.",
    "Berlin has over 2,500 public parks and gardens.",
    "The Victory Column (Siegessäule) is a famous monument in Berlin.",
    "Berlin's Olympic Stadium was built for the 1936 Summer Olympics.",
    "The Berlin State Library is one of the largest libraries in Europe.",
    "The Berlin Dungeon is a popular tourist attraction that offers a spooky look at the city's history.",
    "Berlin's economy is based on high-tech industries and the service sector.",
    "Berlin is a major center for culture, politics, media, and science.",
    "The Berlin Wall Memorial commemorates the division of Berlin and the victims of the Wall.",
    "The city has a large Turkish community, with many residents of Turkish descent.",
    "Berlin's Mauerpark is a popular park known for its flea market and outdoor karaoke sessions.",
    "The Berlin Zoological Garden is the oldest zoo in Germany, opened in 1844.",
    "Berlin is known for its diverse culinary scene, including many vegan and vegetarian restaurants.",
    "The Berliner Dom is a baroque-style cathedral located on Museum Island.",
    "The DDR Museum in Berlin offers interactive exhibits about life in East Germany.",
    "Berlin has a strong cycling culture, with many dedicated bike lanes and bike-sharing programs.",
    "Berlin's Tempodrom is a multi-purpose event venue known for its unique architecture.",
    "The Berlinische Galerie is a museum of modern art, photography, and architecture.",
    "Berlin's Volkspark Friedrichshain is the oldest public park in the city, established in 1848.",
    "The Hackesche Höfe is a complex of interconnected courtyards in Berlin's Mitte district, known for its vibrant nightlife and art scene.",
    "Berlin's International Congress Center (ICC) is one of the largest conference centers in the world."
]

In [2]:
# Install the faiss-cpu library
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 47.5 MB/s eta 0:00:00


In [3]:
# Import Libraries
import faiss
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel

In [4]:
# Initialize the tokenizer and model for generating embeddings
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/paraphrase-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/paraphrase-MiniLM-L6-v2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

In [5]:
# Function to tokenize the input text and generate its embeddings
def embed_text(text, tokenizer, model):
  # Tokenize the input text; return tensors in pytorch; apply padding and truncation
  inputs = tokenizer(text,
                     return_tensors="pt",
                     padding=True,
                     truncation=True)
  # Disable gradient calculations
  with torch.no_grad():
    # Pass the tokenized input through the model to get the state
    embeddings = model(**inputs).last_hidden_state

    # Get the embeddings from the model
    embeddings = embeddings.mean(dim = 1)

  return embeddings

In [6]:
# Initialize a list to store the document embeddings
document_embeddings = []

# Loop through the documents to compute the embeddings
for doc in documents:
  # Embed the current document
  doc_embedding = embed_text(doc, tokenizer, model)

  # and add it to the list
  document_embeddings.append(doc_embedding)

In [7]:
# Concatenate all embeddings into a tensor (pytorch), move it to the CPU, and then convert to array
document_embeddings = torch.cat(document_embeddings).cpu().numpy()
document_embeddings[:1]

array([[ 4.46767867e-01,  1.78271100e-01, -7.59798437e-02,
        -2.06922755e-01,  1.39288262e-01,  3.12733650e-01,
         3.13279867e-01,  1.79999322e-01, -2.68571377e-01,
         1.05027571e-01,  1.02402441e-01, -2.33299240e-01,
         1.21055447e-01, -8.10882375e-02,  9.37905535e-02,
        -3.15877020e-01,  5.29508181e-02,  6.14594519e-02,
         3.22916247e-02,  1.97211709e-02, -4.35076132e-02,
        -2.42606133e-01,  3.20975691e-01, -4.68969017e-01,
         5.46749175e-01, -1.45347625e-01,  2.46898934e-01,
         2.63592154e-01, -2.69205701e-02,  4.53849524e-01,
         4.39478844e-01, -2.98493709e-02,  3.30655843e-01,
        -5.94202504e-02,  4.39280234e-02, -2.56125897e-01,
         6.22930944e-01, -1.07442461e-01, -1.89886466e-01,
         1.79668635e-01, -1.32382825e-01,  2.58974403e-01,
        -2.40420938e-01,  1.59991398e-01,  3.80441159e-01,
         4.35713112e-01,  3.52423638e-01,  2.40945145e-01,
        -1.44617096e-01, -1.41249716e-01,  3.06132495e-0

In [8]:
document_embeddings.shape[1]

384

# Initialize the FAISS index for L2 (Euclidean) distance


In [9]:
# Create an index with dimension equal to the size of the document embeddings
index = faiss.IndexFlatL2(document_embeddings.shape[1])

# Add the document embeddings to the FAISS index for similarity search
index.add(document_embeddings)

In [10]:
# Retrieval function to find and return the most similar documents to the query
def retrieve(query, tokenizer, model, index, documents, top_k=3):
    # Generate the embedding for the query using the provided tokenizer and model
    query_embedding = embed_text(query, tokenizer, model)

    # Search the FAISS index for the top_k most similar documents
    distances, indices = index.search(query_embedding, top_k)

    # Return the most similar documents and their corresponding distances
    return [documents[i] for i in indices[0]], distances[0]

In [11]:
# Test the retrieval function with a sample query
query = "What is the capital of Germany?"

# Retrieve the most similar documents and their distances using the retrieval function
retrieved_docs, distances = retrieve(query, tokenizer, model, index, documents)

# Print the retrieved documents
print(retrieved_docs)
# Print the distances of the retrieved documents from the query
print(distances)

['Berlin is the capital and largest city of Germany by both area and population.', 'Berlin is a major center for culture, politics, media, and science.', 'Berlin is known for its art scene and modern landmarks like the Berliner Philharmonie.']
[36.253918 45.818707 48.38138 ]


# Load the GPT-2 tokenizer and model for generating text

In [12]:
# Import the AutoModelForCausalLM class for generative models
from transformers import AutoModelForCausalLM
gen_tokenizer = AutoTokenizer.from_pretrained("gpt2")
gen_model = AutoModelForCausalLM.from_pretrained("gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [13]:
# Set the pad token to the EOS token
gen_tokenizer.pad_token = gen_tokenizer.eos_token

In [14]:
# Define the context
context = " ".join(retrieved_docs)
context

'Berlin is the capital and largest city of Germany by both area and population. Berlin is a major center for culture, politics, media, and science. Berlin is known for its art scene and modern landmarks like the Berliner Philharmonie.'

In [15]:
# Define a function to generate text based on context and query
def generate_text(context, query, model, tokenizer, max_length = 100):
    # Format the input text with context and query
    input_text = f"Context {context}\nQuestion: {query}\nAnswer:"

    # Tokenize the input text and prepare tensors for the model
    inputs = tokenizer(input_text,
                       return_tensors="pt",
                       padding=True,
                       truncation=True)

    input_ids = inputs['input_ids']
    attention_masks = (input_ids != tokenizer.pad_token_id).long()

    # Generate text using the model
    outputs = model.generate(
        input_ids,
        attention_mask=attention_masks,
        max_length=max_length,
        pad_token_id=tokenizer.eos_token_id
    )

    # Decode the generated text to a readable format
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [16]:
# Define the RAG function than integrates retrieval and generation
def rag(query, retrieval_tokenizer, retrieval_model, retrival_index, gen_model, gen_tokenizer, documents, top_k):
  # Retrieve the most relevant documents for the query
  retrieved_docs, distances = retrieve(query,
                                       retrieval_tokenizer,
                                       retrieval_model,
                                       retrival_index,
                                       documents,
                                       top_k)

  # Combine retrieved documents into a single context
  context = " ".join(retrieved_docs)
  # Generate an answer based on the context and query
  generated_answer = generate_text(context, query, gen_model, gen_tokenizer)

  return generated_answer

In [17]:
# Text the RAG System
query = "What is Berlin famous for?"
answer = rag(query, tokenizer, model, index, gen_model, gen_tokenizer, documents, 3)
print(f"Answer: {answer}")

Answer: Context Berlin is known for its art scene and modern landmarks like the Berliner Philharmonie. Berlin is a major center for culture, politics, media, and science. Berlin is famous for its diverse architecture, ranging from historic buildings to modern structures.
Question: What is Berlin famous for?
Answer: Berlin is famous for its art scene and modern landmarks like the Berliner Philharmonie. Berlin is famous for its diverse architecture, ranging from historic buildings to modern structures.
Question


In [18]:
# Test the RAG System with multiple queries
queries = ["What is the capital of Germany?",
           "What is Berlin famous for?",
           "Who discovered the way to Brazil?",
           "What is the most famous person in Lesotho?"]

In [20]:
for query in queries:
  # Retrieve documents and distances for each query
  retrived_docs, distances = retrieve(query,
                                      tokenizer,
                                      model,
                                      index,
                                      documents,
                                      top_k = 3)
  # Print the query
  print(f"Query: {query}")

  # Print the distances of retrieved documents
  print(f"Distances: {distances}")

  # Print the retrieved documents
  print(f"Documents: {retrieved_docs}")

  # Print a blank line for readability
  print()

Query: What is the capital of Germany?
Distances: [36.253918 45.818707 48.38138 ]
Documents: ['Berlin is the capital and largest city of Germany by both area and population.', 'Berlin is a major center for culture, politics, media, and science.', 'Berlin is known for its art scene and modern landmarks like the Berliner Philharmonie.']

Query: What is Berlin famous for?
Distances: [30.655811 34.762123 35.3083  ]
Documents: ['Berlin is the capital and largest city of Germany by both area and population.', 'Berlin is a major center for culture, politics, media, and science.', 'Berlin is known for its art scene and modern landmarks like the Berliner Philharmonie.']

Query: Who discovered the way to Brazil?
Distances: [70.436806 77.737144 77.84606 ]
Documents: ['Berlin is the capital and largest city of Germany by both area and population.', 'Berlin is a major center for culture, politics, media, and science.', 'Berlin is known for its art scene and modern landmarks like the Berliner Philha

# Define a function to check document relevance based on distance

In [21]:
# Documents with distance below the threshold are considered relevant
def is_relevant(distance , threshold =40):
  return distance < threshold

In [22]:
def rag(query, retrieval_tokenizer, retrieval_model, retrival_index, gen_model, gen_tokenizer, documents, top_k):
  # Retrieve relevant documents and their distances
  retrieved_docs, distances = retrieve(query,
                                       retrieval_tokenizer,
                                       retrieval_model,
                                       retrival_index,
                                       documents,
                                       top_k)

  # Filter out documents that do not meet the relevance criteria
  relevant_docs = [doc for doc, distance in zip(retrieved_docs, distances) if is_relevant(distance, 40)]

  # Return a message if no relevant documents are found
  if not relevant_docs:
    return "I am sorry, there is no relevant information"

  # Combine relevant documents into a context
  context = " ".join(relevant_docs)
  # Generate an answer based on the context and query
  generated_answer = generate_text(context, query, gen_model, gen_tokenizer)

  return generated_answer

In [23]:
# Test the RAG system
for query in queries:
  # Get the generated answer for each query using the RAG function
  answer = rag(query,
               tokenizer,
               model, index,
               gen_model,
               gen_tokenizer,
               documents,
               3)

  # Print the query and the corresponding generated answer
  print(f"Query: {query}\nAnswer: {answer}\n")

Query: What is the capital of Germany?
Answer: Context Berlin is the capital and largest city of Germany by both area and population.
Question: What is the capital of Germany?
Answer: Berlin is the capital of Germany.
Question: What is the capital of Germany?
Answer: Berlin is the capital of Germany.
Question: What is the capital of Germany?
Answer: Berlin is the capital of Germany.
Question: What is the capital of Germany?
Answer: Berlin is the capital of Germany.
Question: What

Query: What is Berlin famous for?
Answer: Context Berlin is known for its art scene and modern landmarks like the Berliner Philharmonie. Berlin is a major center for culture, politics, media, and science. Berlin is famous for its diverse architecture, ranging from historic buildings to modern structures.
Question: What is Berlin famous for?
Answer: Berlin is famous for its art scene and modern landmarks like the Berliner Philharmonie. Berlin is famous for its diverse architecture, ranging from historic buildi

#   Generate text using the model with specified parameters

In [28]:
# Define a function to generate text based on context and query
def generate_text(context, query, model, tokenizer, max_length = 100):
  # Format the input text with context and query
  input_text = f"Context {context}\nQuestion: {query}\nAnswer:"

  # Tokenize the input text and prepare tensors for the model
  inputs = tokenizer(input_text,
                    return_tensors = "pt",
                    padding = True,
                    truncation = True)
  input_ids = inputs['input_ids']
  attention_masks = (input_ids != tokenizer.pad_token_id).long()

  # Generate text using the model with specified parameters
  outputs = gen_model.generate(
      input_ids,
      attention_mask = attention_masks,
      max_length = 100,
      pad_token_id = tokenizer.eos_token_id,
      temperature=0.1,  # Controls randomness in generation
      top_k=50,         # Limits sampling to the top_k most likely tokens
      top_p=0.8,        # Uses nucleus sampling to consider tokens with cumulative probability top_p
      repetition_penalty=1.2,  # Penalizes repeated tokens
      do_sample=True    # Enables sampling
  )

  # Decode the generated text to a readable format
  return tokenizer.decode(outputs[0], skip_special_tokens = True)

In [29]:
# Define the RAG function than integrates retrieval and generation
def rag(query, retrieval_tokenizer, retrieval_model, retrival_index, gen_model, gen_tokenizer, documents, top_k):
  # Retrieve relevant documents and their distances
  retrieved_docs, distances = retrieve(query,
                                       retrieval_tokenizer,
                                       retrieval_model,
                                       retrival_index,
                                       documents,
                                       top_k)

  # Filter out documents that do not meet the relevance criteria
  relevant_docs = [doc for doc, distance in zip(retrieved_docs, distances) if is_relevant(distance, 40)]

  # Return a message if no relevant documents are found
  if not relevant_docs:
    return "I am sorry, there is no relevant information"

  # Combine relevant documents into a context
  context = " ".join(relevant_docs)

  # Generate an answer based on the context and query
  generated_answer = generate_text(context, query, gen_model, gen_tokenizer)

  return generated_answer

In [30]:
# Test the RAG system
for query in queries:
  # Get the generated answer for each query using the RAG function
  answer = rag(query,
               tokenizer,
               model, index,
               gen_model,
               gen_tokenizer,
               documents,
               3)

  # Print the query and the corresponding generated answer
  print(f"Query: {query}\nAnswer: {answer}\n")

Query: What is the capital of Germany?
Answer: Context Berlin is the capital and largest city of Germany by both area and population.
Question: What is the capital of Germany?
Answer: The German state has a total GDP per capita (GDP) that exceeds $1,000 in some countries but only about half as much at other places such Aschweig-Belsen or Schleswigberg . In fact, it's not even close to this level for most European states with large populations like France , Italy and Spain - which

Query: What is Berlin famous for?
Answer: Context Berlin is known for its art scene and modern landmarks like the Berliner Philharmonie. Berlin is a major center for culture, politics, media, and science. Berlin is famous for its diverse architecture, ranging from historic buildings to modern structures.
Question: What is Berlin famous for?
Answer: The city has been named after German poet Heinrich Himmler who wrote "The Art of War". In 1848 he was assassinated by his own government in order that Germany coul